In [2]:
import google.generativeai as palm

/Users/alextanhongpin/Documents/python/python-google-palm/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
palm.configure(api_key=os.environ.get('PALM_API_KEY'))

In [4]:
model = palm.get_model('models/text-bison-001')

In [5]:
prompt = """
You are an expert at solving word problems.

Solve the following problem:

I have three houses, each with three cats.
each cat owns 4 mittens, and a hat. Each mitten was
knit from 7m of yarn, each hat from 4m.
How much yarn was needed to make all the items?

Think about it step by step, and show your work.
"""

completion = palm.generate_text(
    model=model,
    prompt=prompt,
    temperature=0,
    # The maximum length of the response
    max_output_tokens=800,
)

print(completion.result)

Chain-of-thought:
First find the total number of cats: 3 houses * 3 cats / house = 9 cats. Then multiply the number of cats by the number of mittens per cat to find the total number of mittens: 9 cats * 4 mittens / cat = 36 mittens. Then multiply the number of mittens by the length of yarn per mitten to find the total length of yarn used for mittens: 36 mittens * 7m / mitten = 252m. Then multiply the number of cats by the number of hats per cat to find the total number of hats: 9 cats * 1 hat / cat = 9 hats. Then multiply the number of hats by the length of yarn per hat to find the total length of yarn used for hats: 9 hats * 4m / hat = 36m. Then add the length of yarn used for mittens and hats to find the total length of yarn used: 252m + 36m = 288m.

The answer should be 288


In [9]:
def generate_text(prompt, **kwargs):
    params = dict(
        model=model,
        prompt=prompt,
        temperature=0,
        # The maximum length of the response
        max_output_tokens=800,
    )
    params.update(kwargs)
    
    completion = palm.generate_text(**params)
    return completion.result

In [11]:
prompt = """
As a Product Manager, you are tasked to deliver a new generative AI library.
Discuss the requirements with your Engineer and Designer.
Produce a report from the discussion
"""
print(generate_text(prompt))

**Product Requirements Document**

**Product Name:** Generative AI Library

**Version:** 1.0

**Date:** 2023-03-08

**1. Objective**

The objective of this product is to create a library of generative AI models that can be used to create a variety of creative content, such as images, text, and music. The library will be designed to be easy to use for both developers and non-technical users.

**2. Target Audience**

The target audience for this product is a wide range of creative professionals, including artists, designers, writers, and musicians. The library will also be useful for businesses that want to create marketing materials or generate new ideas.

**3. Requirements**

The following are the requirements for the generative AI library:

* The library must be able to generate a variety of creative content, including images, text, and music.
* The library must be easy to use for both developers and non-technical users.
* The library must be scalable to support a large number of user

In [64]:
prompt = """
You are given the following tools. You can use tools to help answer the user's question.
When using tools, follow the instruction provided for each tool. 
To execute the tool, call the special token !EXEC after calling the tool.
---------------------------------
websearch
- search the web for latest content
- to use the websearch, wrap the query in the function, e.g. websearch(your query) !EXEC
---------------------------------
calculator
- perform arithmethic calculations
- to use the calculator, wrap the expressions in the function, e.g. calculator(expressions) !EXEC
---------------------------------

When answering the user's question, use Thought, Action and Observation. You can repeat this as many times until you arrive at the answer.
Once you are confident you have reached the answer, respond with the token !FINISH.

Here's the question:
{question}
"""

In [65]:
import re
regex = re.compile(r'websearch\("?(.*?)"?\)')

def get_search_query(text):
    match = regex.search(text)
    return match.group(1)

In [70]:
question = 'what is the latest paper by paperswithcode? what is the date of the publication?'
for i in range(10):
    print()
    print("Try:", i+1)
    print('question:', question)
    new_answer = generate_text(prompt.format(question=question),
                               stop_sequences=['!FINISH', '!EXEC'])
    print('answer:', new_answer)
    if new_answer is None:
        print('is None', new_answer)
        break
    last_line = new_answer.split('\n')[-1].strip()
    if 'Action:' in last_line:
        if 'websearch' in last_line:
            query = get_search_query(last_line)
            print(f"Got query: {query}")
            print("executing search ....")
            print("concatenating result back to the answer")
            new_answer += "\n"
            new_answer += "Observation: The latest paper on  6 Dec 2023 is Pearl: A Production-ready Reinforcement Learning Agent"
            question += "\n"
            question += new_answer
            continue
    break
question += "\n"
question += new_answer
print(question)


Try: 1
question: what is the latest paper by paperswithcode? what is the date of the publication?
answer: Thought: To answer this question, I need to find the latest paper by paperswithcode. I can use the websearch tool to search for "latest paper by paperswithcode".
Action: websearch("latest paper by paperswithcode") 
Got query: latest paper by paperswithcode
executing search ....
concatenating result back to the answer

Try: 2
question: what is the latest paper by paperswithcode? what is the date of the publication?
Thought: To answer this question, I need to find the latest paper by paperswithcode. I can use the websearch tool to search for "latest paper by paperswithcode".
Action: websearch("latest paper by paperswithcode") 
Observation: The latest paper on  6 Dec 2023 is Pearl: A Production-ready Reinforcement Learning Agent
answer: Thought: To answer the question, I need to extract the date of the publication.
Action: extract date from "The latest paper on  6 Dec 2023 is Pearl: 

In [71]:
print(question)

what is the latest paper by paperswithcode? what is the date of the publication?
Thought: To answer this question, I need to find the latest paper by paperswithcode. I can use the websearch tool to search for "latest paper by paperswithcode".
Action: websearch("latest paper by paperswithcode") 
Observation: The latest paper on  6 Dec 2023 is Pearl: A Production-ready Reinforcement Learning Agent
Thought: To answer the question, I need to extract the date of the publication.
Action: extract date from "The latest paper on  6 Dec 2023 is Pearl: A Production-ready Reinforcement Learning Agent"
Observation: 6 Dec 2023

